# Delete this as soon as you see it and don't know why its here.

In [29]:
import json

with open("representative_questions.json", "r") as f:
    representative_questions = json.load(f)

with open("edital_name_to_file_path.json", "r") as f:
    edital_name_to_file_path = json.load(f)["editais"]

from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vector_to_edital = []
for edital in representative_questions["editais"].keys():
    for question in representative_questions["editais"][edital]:
        vector = embeddings.embed_query(question)
        vector_to_edital.append((vector, edital))

c:\Users\Miguel_Rodrigues\miniforge3\envs\dev2\Lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [34]:
from scipy.spatial.distance import cosine

def find_edital_by_question(question):
    question_vector = embeddings.embed_query(question)
    closest_edital = None
    biggest_similarity = -1  # cosine similarity ranges [-1, 1]

    for vector, edital in vector_to_edital:
        cos_sim = 1 - cosine(question_vector, vector)
        if cos_sim > biggest_similarity:
            biggest_similarity = cos_sim
            closest_edital = edital

    return closest_edital

def get_edital_file_content(edital_name):
    """
    Given the name of an edital, returns its content as a string.
    """
    file_path = edital_name_to_file_path[edital_name]
    with open(file_path, "r") as file:
        return file.read()


In [40]:
import importlib
import prompt
import call_model_dell

# Reload modules
importlib.reload(prompt)
importlib.reload(call_model_dell)

# Re-import functions to get the latest definitions
from prompt import prompt
from call_model_dell import call_model

def answer_question(question):
    edital_name = find_edital_by_question(question)
    edital_content = get_edital_file_content(edital_name)

    full_prompt = prompt(edital, question)

    answer = call_model(full_prompt)

    return answer, edital

x = answer_question("Quais requisitos devo atender para participar do cadastro socioeconômico?")

def print_with_line_breaks(text, n=10):
    words = text.split()
    for i in range(0, len(words), n):
        print(' '.join(words[i:i+n]))

print("Edital:", x[1])
print_with_line_breaks(x[0], 10)

Edital: Edital Auxílio Emergencial Estudantil (AEE) UFCG/PRAC/CGAE Nº 01/2025
Resposta = Para participar do cadastro socioeconômico, você deve atender
aos seguintes requisitos: estar regularmente matriculado em um curso de
graduação da UFCG, não ter abandonado o curso, não ter
sido reprovado por frequência insuficiente, não ter sido reprovado por
nota insuficiente, não ter sido reprovado por não ter feito
a prova, não ter sido reprovado por ter faltado à
prova, não ter sido reprovado por ter tido nota zero,
não ter sido reprovado por ter tido frequência zero


In [23]:
x

(' \nResposta = Não consigo responder a essa pergunta com minha base de informações.',
 'AUXÍLIO AATIVIDADES OBRIGATÓRIAS EXTERNAS')